In [1]:
import numpy as np
import torch
import torch.nn as nn
import tensorflow as tf
from tqdm import tqdm
!pip -q install datasets transformers
import datasets
from datasets import load_dataset
from transformers import BertForSequenceClassification, BertTokenizer,default_data_collator,get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
tf.random.set_seed(seed)

In [3]:
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-small",num_labels=2)
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-small")
dataset = load_dataset("sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/116M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
#VARS
num_epochs = 7
lrn = 3e-2
BATCH = 128
max_length = 300

In [5]:
#FUNCTIONS

def preprocess_function(examples):
    inputs = examples['sentence']
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf",return_attention_mask=True)
    # model_inputs = {
    #     'input_ids': tf.convert_to_tensor(model_inputs['input_ids'])
    #  }
    model_inputs["labels"] = examples['label']
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    # remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/67349 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/872 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=BATCH, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=BATCH, pin_memory=True)

In [7]:
class Prompt_model(nn.Module):
  def __init__(self,model,prompt_init):
    super().__init__()
    self.model = model.to("cuda")
    print("bert params to be trained:-")
    for name,param in self.model.named_parameters():
      if(name[0] != "c"):
        param.requires_grad = False
      else:
        print(name)
    tokens = tokenizer(prompt_init,padding=True,return_tensors='pt',truncation=True,return_attention_mask=True).to("cuda")
    prompt_embedds = self.model.get_input_embeddings()(tokens['input_ids'])
    self.prompt = torch.from_numpy(prompt_embedds.cpu().numpy())[0].to("cuda")
    self.att = tokens['attention_mask']
    self.token_id = tokens['token_type_ids']
    # print(self.att.shape)
    self.get_prompt = nn.Sequential(
        nn.Linear(512,128),
        nn.ReLU(),
        nn.Linear(128,96),
        nn.ReLU(),
        nn.Linear(96,128),
        nn.ReLU(),
        nn.Linear(128,512)
    )

  def forward(self, inputs):
    token_embedds = self.model.get_input_embeddings()(inputs["input_ids"])
    # token_embedds = torch.from_numpy(token_embedds.numpy()).to("cuda")
    stacked_prompts = torch.stack([self.get_prompt(self.prompt)] * np.shape(token_embedds)[0], dim=0)
    stacked_attention = self.att.repeat(np.shape(token_embedds)[0], 1)
    stacked_ids = self.token_id.repeat(np.shape(token_embedds)[0],1)
    # print(self.att.shape)
    # print(stacked_attention.shape)
    # print(type(token_embedds))
    combined_embedds = torch.cat((stacked_prompts,token_embedds),1)
    combined_att = torch.cat((stacked_attention,inputs["attention_mask"]),1)
    combined_token_type_ids = torch.cat((stacked_ids,inputs['token_type_ids']),1)
    features = self.model(inputs_embeds=combined_embedds,labels=inputs["labels"],attention_mask = combined_att,token_type_ids = combined_token_type_ids)
    features.logits = torch.exp(features.logits)
    my_sum = torch.sum(features.logits,dim=1, keepdim=True)
    features.logits = features.logits/my_sum
    # x3 = torch.exp(torch.tensor(features))

    return features

In [8]:
prompt_init = "So nice movie - positive; the movie was bad - negative"
my_model = Prompt_model(model,prompt_init)
optimizer = torch.optim.AdamW(my_model.parameters(), lr=lrn)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


bert params to be trained:-
classifier.weight
classifier.bias


In [9]:
# tokens = tokenizer(prompt_init,padding=True,return_tensors='pt',truncation=True)
# prompt_embedds = model.get_input_embeddings()(tokens['input_ids'])
# model(inputs_embeds=prompt_embedds)
# dir(model.forward)

In [10]:
device = "cuda"
my_model = my_model.to(device)

for epoch in range(num_epochs):
    my_model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        # break
        outputs = my_model.forward(batch)
        # print(outputs)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    # break
    my_model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = my_model.forward(batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 7/7 [00:03<00:00,  2.13it/s]


epoch=0: train_ppl=tensor(1.7949, device='cuda:0') train_epoch_loss=tensor(0.5849, device='cuda:0') eval_ppl=tensor(1.8443, device='cuda:0') eval_epoch_loss=tensor(0.6121, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  1.96it/s]


epoch=1: train_ppl=tensor(1.6236, device='cuda:0') train_epoch_loss=tensor(0.4846, device='cuda:0') eval_ppl=tensor(1.6340, device='cuda:0') eval_epoch_loss=tensor(0.4911, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  2.16it/s]


...

epoch=4: train_ppl=tensor(1.5276, device='cuda:0') train_epoch_loss=tensor(0.4237, device='cuda:0') eval_ppl=tensor(1.5391, device='cuda:0') eval_epoch_loss=tensor(0.4312, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  1.96it/s]


epoch=5: train_ppl=tensor(1.5099, device='cuda:0') train_epoch_loss=tensor(0.4121, device='cuda:0') eval_ppl=tensor(1.5162, device='cuda:0') eval_epoch_loss=tensor(0.4162, device='cuda:0')


100%|██████████| 7/7 [00:03<00:00,  2.13it/s]

epoch=6: train_ppl=tensor(1.4937, device='cuda:0') train_epoch_loss=tensor(0.4013, device='cuda:0') eval_ppl=tensor(1.5082, device='cuda:0') eval_epoch_loss=tensor(0.4109, device='cuda:0')


In [11]:
correct = 0
total = 0
for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = my_model(batch)
            x = outputs.logits
            x = torch.exp(x)
            x = x/torch.sum(x,1,keepdim=True)
            for i in range(x.size()[0]):
              if(x[i][0] < x[i][1] and batch['labels'][i] == 1):
                correct += 1
              elif(x[i][0] > x[i][1] and batch['labels'][i] == 0):
                correct += 1
              total += 1
            # print(batch['labels'])
            # break
print("\n the accuracy is:-")
print(correct * 100 /total)

100%|██████████| 7/7 [00:03<00:00,  2.21it/s]


 the accuracy is:-
81.07798165137615


In [12]:
print("Number of Trainable Params:-")
print(sum(p.numel() for p in my_model.parameters() if p.requires_grad))

Number of Trainable Params:-
157538
